In [113]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import pandas as pd

In [76]:


# Obtener la fecha de inicio del usuario
#fecha_inicio_str = input("Ingrese la fecha de inicio (DD-MM-YYYY): ")
#fecha_inicio = datetime.strptime(fecha_inicio_str, "%d-%m-%Y")

# Obtener la fecha de fin del usuario
#fecha_fin_str = input("Ingrese la fecha de fin (DD-MM-YYYY): ")
#fecha_fin = datetime.strptime(fecha_fin_str, "%d-%m-%Y")

# Definir las dos fechas de inicio y fin
fecha_inicio = datetime(2022, 2, 1)
fecha_fin = datetime(2022, 2, 3)

# obtener primera fecha
link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+fecha_inicio.strftime("%d-%m-%Y")+".csv"
df = pd.read_csv(link, index_col=0)

# sumar un dia
fecha_inicio += timedelta(days=1)

# Recorrer todas las fechas en el intervalo
while fecha_inicio <= fecha_fin:

    link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+fecha_inicio.strftime("%d-%m-%Y")+".csv"
    df2 = pd.read_csv(link,index_col=0)
    df = pd.concat([df,df2])
    fecha_inicio += timedelta(days=1)

df.reset_index(inplace=True)
df


   




,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-03 02:22:25,33.93911,67.709953,158189,7361,NaN,NaN,Afghanistan,406.359470,4.653294
1,NaN,NaN,NaN,Albania,2022-01-03 02:22:25,41.15330,20.168300,210885,3220,NaN,NaN,Albania,7327.993606,1.526899
2,NaN,NaN,NaN,Algeria,2022-01-03 02:22:25,28.03390,1.659600,219159,6291,NaN,NaN,Algeria,499.780587,2.870519
3,NaN,NaN,NaN,Andorra,2022-01-03 02:22:25,42.50630,1.521800,23740,140,NaN,NaN,Andorra,30725.425484,0.589722
4,NaN,NaN,NaN,Angola,2022-01-03 02:22:25,-11.20270,17.873900,82920,1772,NaN,NaN,Angola,252.295150,2.137000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12043,NaN,NaN,Unknown,Ukraine,2022-03-03 04:21:01,NaN,NaN,39398,734,0.0,0.0,"Unknown, Ukraine",0.000000,0.000000
12044,NaN,NaN,NaN,Nauru,2022-03-03 04:21:01,-0.52280,166.931500,0,0,0.0,0.0,Nauru,0.000000,0.000000
12045,NaN,NaN,Niue,New Zealand,2022-03-03 04:21:01,-19.05440,-169.867200,0,0,0.0,0.0,"Niue, New Zealand",0.000000,0.000000
12046,NaN,NaN,NaN,Tuvalu,2022-03-03 04:21:01,-7.10950,177.649300,0,0,0.0,0.0,Tuvalu,0.000000,0.000000


In [125]:
filas = str(df.shape[0])
columna = str(df.shape[1])

df[["Active","Recovered"]] = df[["Active","Recovered"]].astype("int")


print("el conjunto de dato tiene "+filas+" filas y "+columna+" columnas")
print("")

print("")
print("Tipos de datos")
print(df.dtypes)
print("")



el conjunto de dato tiene 12048 filas y 14 columnas


Tipos de datos
FIPS                   float64
Admin2                  object
Province_State          object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                   int64
Recovered                int32
Active                   int32
Combined_Key            object
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object



In [120]:
df["Recovered"].unique()

array([0.])

In [100]:

datos_faltantes = df.isnull()
print("datos faltantes")
for columna in datos_faltantes.columns.values.tolist():
    print('columna:',columna)
    print (datos_faltantes[columna].value_counts())
    print("")    

datos faltantes
columna: FIPS
FIPS
False    9804
True     2244
Name: count, dtype: int64

columna: Admin2
Admin2
False    9816
True     2232
Name: count, dtype: int64

columna: Province_State
Province_State
False    11511
True       537
Name: count, dtype: int64

columna: Country_Region
Country_Region
False    12048
Name: count, dtype: int64

columna: Last_Update
Last_Update
False    12048
Name: count, dtype: int64

columna: Lat
Lat
False    11775
True       273
Name: count, dtype: int64

columna: Long_
Long_
False    11775
True       273
Name: count, dtype: int64

columna: Confirmed
Confirmed
False    12048
Name: count, dtype: int64

columna: Deaths
Deaths
False    12048
Name: count, dtype: int64

columna: Recovered
Recovered
True     12022
False       26
Name: count, dtype: int64

columna: Active
Active
True     12022
False       26
Name: count, dtype: int64

columna: Combined_Key
Combined_Key
False    12048
Name: count, dtype: int64

columna: Incident_Rate
Incident_Rate
False    117

In [126]:
# 4. Número de casos confirmados, fallecidos, recuperados y activos por país/región
df["Recovered"].replace(np.nan,0,inplace=True)
df["Active"].replace(np.nan,0,inplace=True)

dead = df.groupby("Country_Region").sum()
dead[["Confirmed","Deaths","Recovered","Active"]]


,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,496452,22396,0,0
Albania,743950,10056,0,0
Algeria,739123,19738,0,0
Andorra,98304,437,0,0
Angola,279933,5567,0,0
...,...,...,...,...
West Bank and Gaza,1663963,15567,0,0
Winter Olympics 2022,828,0,0,0
Yemen,33014,6137,0,0


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-03 02:22:25,33.93911,67.709953,158189,7361,NaN,NaN,Afghanistan,406.359470,4.653294
4016,NaN,NaN,NaN,Afghanistan,2022-02-03 04:21:21,33.93911,67.709953,164190,7418,NaN,NaN,Afghanistan,421.774974,4.517937
8032,NaN,NaN,NaN,Afghanistan,2022-03-03 04:21:01,33.93911,67.709953,174073,7617,NaN,NaN,Afghanistan,447.162647,4.375750
